<a href="https://www.kaggle.com/code/lonnieqin/software-defects-prediction-with-lgbm?scriptVersionId=148131127" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Software Defects Prediction with LGBM

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e23/train.csv")
train.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [3]:
feature_columns = list(train.columns)
feature_columns.remove("id")
target_column = "defects"
feature_columns.remove(target_column)

In [4]:
models =[]
kfold = RepeatedStratifiedKFold(n_repeats=5, n_splits=10, random_state=2024)
for train_indices, valid_indices in kfold.split(train, train[target_column]):
    model = lgb.LGBMClassifier()
    model.fit(
        train.iloc[train_indices][feature_columns], 
        train.iloc[train_indices][target_column], 
        eval_set=(train.iloc[valid_indices][feature_columns], train.iloc[valid_indices][target_column])
    )
    models.append(model)

[1]	valid_0's binary_logloss: 0.513959
[2]	valid_0's binary_logloss: 0.497824
[3]	valid_0's binary_logloss: 0.485036
[4]	valid_0's binary_logloss: 0.475006
[5]	valid_0's binary_logloss: 0.466841
[6]	valid_0's binary_logloss: 0.460207
[7]	valid_0's binary_logloss: 0.454793
[8]	valid_0's binary_logloss: 0.450249
[9]	valid_0's binary_logloss: 0.446513
[10]	valid_0's binary_logloss: 0.443434
[11]	valid_0's binary_logloss: 0.440835
[12]	valid_0's binary_logloss: 0.438682
[13]	valid_0's binary_logloss: 0.43681
[14]	valid_0's binary_logloss: 0.435432
[15]	valid_0's binary_logloss: 0.434116
[16]	valid_0's binary_logloss: 0.433028
[17]	valid_0's binary_logloss: 0.432215
[18]	valid_0's binary_logloss: 0.431454
[19]	valid_0's binary_logloss: 0.430746
[20]	valid_0's binary_logloss: 0.430257
[21]	valid_0's binary_logloss: 0.42981
[22]	valid_0's binary_logloss: 0.429367
[23]	valid_0's binary_logloss: 0.429119
[24]	valid_0's binary_logloss: 0.42881
[25]	valid_0's binary_logloss: 0.42857
[26]	valid_0'

In [5]:
test = pd.read_csv("/kaggle/input/playground-series-s3e23/test.csv")
test["defects"] = np.mean([model.predict_proba(test[feature_columns])[:, 1] for model in models], axis=0)
test[["id", "defects"]].to_csv("submission.csv", index=False)
test[["id", "defects"]].head()

,id,defects
0,101763,0.248379
1,101764,0.182167
2,101765,0.655619
3,101766,0.462481
4,101767,0.140674
